In [3]:
import datetime
import json
import os
import pandas as pd
import logging


In [13]:
from utils.data_transformation import prepare_dialogs, if_in_date_range

In [13]:
!pip install -r requirements.txt

     |████████████████████████████████| 10.3 MB 755 kB/s eta 0:00:01
     |████████████████████████████████| 15.1 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 2.9 MB/s eta 0:00:01
  Using cached num2words-0.5.10-py3-none-any.whl (101 kB)
  Using cached pip-20.2.2-py2.py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 7.2 MB 2.9 MB/s eta 0:00:01     |█████▏                          | 1.2 MB 3.3 MB/s eta 0:00:02     |██████████████████▋             | 4.2 MB 4.4 MB/s eta 0:00:01     |██████████████████████████▏     | 5.9 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 300 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 690 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 28.9 MB 4.4 MB/s eta 0:00:01    |████                            | 3.7 MB 1.4 MB/s eta 0:00:19     |█

You should consider upgrading via the '/Users/akurochkin/testenv/bin/python3.8 -m pip install --upgrade pip' command.


In [12]:
# TODO: add this packages to the requirements.txt file

# !pip install -U nlpcube
# !pip install stop-words
# !pip install git+https://github.com/Desklop/Uk_Stemmer

  Cloning https://github.com/Desklop/Uk_Stemmer to /private/var/folders/b3/dv3g0qqj6z13sqw26_2z04140000gp/T/pip-req-build-u78_wvnb
  Created wheel for uk-stemmer: filename=uk_stemmer-1.0-py3-none-any.whl size=9498 sha256=21815c007406eb80b2344666d439ff325138f81df816aec6c8ef75625025954a
  Stored in directory: /private/var/folders/b3/dv3g0qqj6z13sqw26_2z04140000gp/T/pip-ephem-wheel-cache-e7uvmikz/wheels/40/53/2f/982431dc2aff61a4eb52a030950bbbdef0d9178114373d6c0e
Successfully built uk-stemmer


In [17]:
# TODO: rename this method, calculate is to generic word, name should give you information about method
def calculate_df(dialog_data, date_before, date_after, user_id_get_msg, dialog_id, path_to_static):
    """
    :param dialog_data: a pandas dataframe of dialogs
    :param date_before: datetime type, from what time start to analyse msgs
    :param date_after: datetime type, to what time to analyse msgs
    :param user_id_get_msg: a user chat id, who msgs to analyse or "all" - for all participants of the dialog
    :param dialog_id: str
    :return: a sorted df - sorted dict of user_id_get_msg most used words
    """
    DF = {}
    file_text = ""

    for row in dialog_data.index:
        if user_id_get_msg != "all":
            if dialog_data["from_id"][row] != user_id_get_msg:
                continue

        if if_in_date_range(dialog_data["date"][row][:-6], date_before, date_after) == 'Dialog after date_after':
            continue

        elif if_in_date_range(dialog_data["date"][row][:-6], date_before, date_after):
            if not pd.isnull(dialog_data["message"][row]):
                file_text += dialog_data["message"][row] + " "
                for w in dialog_data["message"][row].split():
                    try:
                        DF[w] += 1
                    except:
                        DF[w] = 1
        else:
            break

    with open(path_to_static + "{}/1_{}.txt".format(dialog_id, dialog_id), "w", encoding="utf-8") as f:
        f.write(file_text)

    DF_sorted = {k: v for k, v in sorted(DF.items(), key=lambda item: item[1], reverse=True)}
    
    return DF_sorted

In [28]:
# 1) change a dialog_id, which you want to investigate
dialog_id = "1355484549"

# 2) change user_id_get_msg
# int type - user chat id, who massages you want to analyse,
# you can find it in <dialog_id>.csv
# str type "all" if you want to analyse msgs of all users of the dialog
user_id_get_msg = 511986933
dialog_path = "data/dialogs/"
prep_path = 'data/prepared_dialogs/'

# TODO: path_to_static, static what? you should rename this folder with respect to its purpose!
path_to_static = "data/tmp_static/"

# TODO: rename date_before and date_after -> start_date and end_date
# 3) change to date range in what you want to analyse messages of user_id_get_msg - from date_before to date_after;
# format "%Y-%m-%d %H:%M:%S"
date_before = datetime.datetime(2020, 7, 9, 0, 0, 0)
date_after = datetime.datetime(2020, 8, 10, 0, 0, 0)

In [29]:
# TODO: add log file path to global var
logging.basicConfig(filename='logs/project_logs.log', level=0)
logging.info("starting logs for tf_idf_dialogs")

# TODO: move instructions in the README file

# 0) in console input: import nltk; nltk.download()
#
# Install (or update) NLP-Cube with:
# pip3 install -U nlpcube
#
# use telegram-data-collection/0_download_dialogs_list.py and 1_download_dialogs_data.py
# to get some files to analyse
# files SHOULD be in data/dialogs and data/dialogs_meta

In [30]:
if not os.path.exists(path_to_static):
    os.mkdir(path_to_static)
    
if not os.path.exists(path_to_static + dialog_id):
    os.mkdir(path_to_static + dialog_id)

In [31]:
# TODO: create a method to detect dialog language

# 4) change "ua" to a language of your dialog ("ua", "ru" or "en")
# if you write "ua" or "ru" - dialog will be cleaned from stop_words_ua + stop_words_ru + stop_words_en
# so do not bother about these languages if you can not write an exactly language of the dialog

# TODO: move lang to the variable
prepare_dialogs(dialog_id, dialog_path, prep_path, date_before,
                date_after, "ru", "words_frequency")

INDEX 146
INDEX 149
INDEX 150
INDEX 151
INDEX 154
INDEX 155
INDEX 156
INDEX 157
INDEX 158
INDEX 161
INDEX 162
INDEX 163
INDEX 164
INDEX 165
INDEX 166
INDEX 169
INDEX 170
INDEX 173
INDEX 174
INDEX 175
INDEX 178
INDEX 179
INDEX 180
INDEX 181
INDEX 182
INDEX 185
INDEX 186
INDEX 187
INDEX 188
INDEX 189
INDEX 192
INDEX 193
INDEX 194
INDEX 195
INDEX 196
INDEX 199
INDEX 200
INDEX 201
INDEX 202
INDEX 203
INDEX 206
INDEX 207
INDEX 209
INDEX 210
INDEX 211
INDEX 212
INDEX 217
INDEX 218
INDEX 226
INDEX 227
INDEX 228
INDEX 231
INDEX 232
INDEX 233
INDEX 234
INDEX 235
INDEX 238
INDEX 239
INDEX 240
INDEX 241
INDEX 242
INDEX 243
INDEX 246
INDEX 247
INDEX 248
INDEX 249
INDEX 250
INDEX 253
INDEX 254
INDEX 255
INDEX 256
INDEX 257
INDEX 258
INDEX 261
INDEX 262
INDEX 263
INDEX 264
INDEX 265
INDEX 268
INDEX 269
INDEX 270
INDEX 271
INDEX 272
INDEX 273
INDEX 279
INDEX 280
INDEX 281
INDEX 284
INDEX 285
INDEX 286
INDEX 287
INDEX 288
INDEX 291
INDEX 292
INDEX 293
INDEX 294
INDEX 295
INDEX 298
INDEX 299
INDEX 300


In [32]:
dialog_data = pd.read_csv(prep_path + dialog_id + ".csv")

DF_sorted = calculate_df(dialog_data, date_before, date_after, user_id_get_msg, dialog_id, path_to_static)

In [33]:
with open(path_to_static + "{}/words_frequency1_{}.json".format(dialog_id, dialog_id), "w", encoding="utf-8") as f:
    json.dump(DF_sorted, f, indent=4, ensure_ascii=False)

# 4) after work of this module look at results in static/<dialog_id> dir
# use command under to make a wordcloud, to understand its parameters use - wordcloud_cli --help
# wordcloud_cli --text static/<dialog_id>/1_<dialog_id>.txt --mask static/logo_telegram.jpg --imagefile static/my_words_nazar2.png
print(DF_sorted)

{}
